In [1]:
import pandas as pd
import re
import pickle
import tfidf_matcher as tm

In [2]:
problematic_urls = [33, 44, 45, 50, 62, 82, 84, 104, 132, 135, 219, 248, 264, 303, 307, 322, 353, 378, 393, 417, 425, 449, 
                    461, 465, 498, 530, 543, 545, 547, 551, 596, 619, 623, 638, 648, 666, 676, 683, 686, 692, 715, 718, 725, 
                    729, 775, 780, 798, 804, 811, 817, 838, 868, 898, 918, 935, 961, 963, 969, 982, 1012, 1020, 1034, 1042, 
                    1044, 1049, 1067, 1069, 1090, 1094, 1095, 1101, 1107, 1122, 1133, 1156, 1157, 1161, 1177, 1187, 1191, 
                    1207, 1213, 1222, 1241, 1245, 1262, 1286, 1287, 1289, 1306, 1351, 1432, 1457, 1474, 1506, 1507, 1515, 
                    1519, 1524, 1525, 1535, 1539, 1550, 1584, 1585, 1599, 1603, 1606, 1616, 1629, 1634, 1641, 1652, 1654, 
                    1655, 1658, 1665, 1673, 1688, 1714, 1729, 1736, 1744, 1788, 1803, 1839, 1843, 1848, 1861, 1866, 1886, 
                    1892, 1900, 1905, 1906, 1911, 1912, 1920, 1921, 1971, 1986, 2001, 2005, 2047, 2051, 2077, 2078, 2079, 
                    2081, 2082, 2094, 2097, 2101, 2114, 2116, 2126, 2137, 2142, 2152, 2174, 2175, 2189, 2200, 2209, 2212, 
                    2215, 2232, 2245, 2250, 2257, 2258, 2260, 2275, 2291, 2313, 2314, 2317, 2329, 2332, 2337, 2338, 2342, 
                    2350, 2357, 2358, 2362, 2364, 2378, 2400, 2403, 2406, 2407, 2410, 2415, 2419, 2425, 2435, 2444, 2449, 
                    2450, 2465, 2473, 2486, 2490, 2491, 2502, 2521, 2530, 2555, 2557, 2560, 2562, 2581, 2596, 2597, 2605, 
                    2606, 2612, 2613, 2620, 2633, 2653, 2657, 2658, 2674, 2676, 2682, 2689, 2704, 2712, 2714, 2716, 2726, 
                    2732, 2734, 2747, 2749, 2763, 2773, 2783, 2786, 2798, 2806, 2811, 2814, 2822, 2825, 2834, 2852, 2872, 
                    2903, 2908, 2925, 2930, 2931, 2938, 2977, 2979, 3005, 3006, 3018, 3032, 3034, 3037, 3040, 3048, 3050, 
                    3061, 3085, 3097, 3098, 3104, 3126, 3139, 3153, 3172, 3182, 3184, 3193, 3194, 3195, 3198, 3200, 3206, 
                    3207, 3211, 3231, 3240, 3248, 3254, 3257, 3259, 3277, 3281, 3296, 3304, 3308, 3311, 3323, 3326, 3377, 
                    3380, 3411, 3415, 3422, 3426, 3442, 3449, 3451, 3457, 3466, 3467, 3473, 3475, 3479, 3480, 3500, 3503, 
                    3505, 3509, 3510, 3512, 3514, 3519, 3544, 3547, 3549, 3550, 3551, 3565, 3575, 3608, 3610, 3611, 3624, 
                    3634, 3635, 3636, 3638, 3662, 3703, 3707, 3710, 3716, 3732, 3739, 3743, 3775, 3797, 3824, 3825, 3837]

In [3]:
with open('../Data/news.pkl', 'rb') as f:
    news = pickle.load(f)
    # 3844 unique authors

muckrack = pd.read_csv('../Data/muckrack_persons_fetchlist.csv')

In [4]:
def get_top(n):
    temp = news[['author', 'article_count']].groupby('author').count().sort_values(['article_count'], ascending = False)
    temp = temp.reset_index().head(n)
    urls = []
    for i in range(len(temp)):
        url = temp.loc[i].author.lower()
        url = re.sub(r'\|.*$', '', url) # remove everything after |
        url = re.sub(r'\,.*$', '', url) # remove evrything after ,
        
        punctuations = '''!()[]{};:'"\,<>./?#$%^&*_~'''
        for x in url.lower(): 
            if x in punctuations: url = url.replace(x, "") # remove punctuations except hyphen
        url = re.sub(r'(\d+)', '', url) # remove numbers
        url = re.sub(r'(--)', '', url)
        url = re.sub(r'(\w+@\w+)', '', url) # remove emails
        url = ' '.join([i.strip() for i in url.split()]) # remove spaces and lowercase
        url = re.sub(r'(\s+)(and)(\s+)(\w*)(\s*)(\w*)', '', url) # remove anything after and
        keywords = ['sa', 'tulsa', 'editor', 'writer', 'world-herald', 'news', 'world', 'richmond', 'times-dispatch', 'new',
                   'hampshire', 'union', 'leader', 'for', 'the', 'state', 'journal', 'correspondant', 'sfgate', 'special',
                   'from', 'the', 'gazette', 'times', 'staff', 'senior', 'dr', 'correspondent', 'by', 'editorial board',
                   'research', 'wire reports', 'security', 'real', 'estate', 'to', 'post', 'and', 'courier', 'policy',
                   'commercial', 'bureau', 'political', 'roanoke', 'college', 'football', 'editorial','democrat-gazette',
                   'arizona', 'daily', 'star', '--hamburg', 'column', 'lincoln', 'managing', 'backstage', 'with', 'sports',
                   'ii', 'iii', 'capitol', 'media', 'services']
        url = ' '.join([i for i in url.split() if i not in keywords]) # remove other keywords
        url = re.sub(r'(@)', '', url) # remove handles
        url = 'https://muckrack.com/' + url.replace(' ', '-').strip('-') # make URL
        urls.append(url)
    temp['Muckrack'] = urls
    return temp

In [5]:
data = get_top(3838)

In [6]:
data

,author,article_count,Muckrack
0,Neil Shaw,523,https://muckrack.com/neil-shaw
1,James Rodger,512,https://muckrack.com/james-rodger
2,Jack Davis,251,https://muckrack.com/jack-davis
3,Adam Wells,223,https://muckrack.com/adam-wells
4,Sophie McCoid,218,https://muckrack.com/sophie-mccoid
...,...,...,...
3833,Levi Sumagaysay,10,https://muckrack.com/levi-sumagaysay
3834,Ben Ashford,10,https://muckrack.com/ben-ashford
3835,Craig Brown,10,https://muckrack.com/craig-brown
3836,Courtney Connley,10,https://muckrack.com/courtney-connley


In [31]:
lookup = list(muckrack.url)
k_matches = 3
ngram_length = 2
for i in problematic_urls:
    original = [data.iloc[i].Muckrack]
    results = tm.matcher(original, lookup, k_matches, ngram_length)
    print("To match:", original)
    print("Highest match:", results['Lookup 1'].iloc[0])
    print("Confidence:", results['Match Confidence'].iloc[0])
    print()

To match: ['https://muckrack.com/matthew-cooper']
Highest match: https://muckrack.com/matthew-cooper-3
Confidence: 0.89

To match: ['https://muckrack.com/seamus-duff']
Highest match: https://muckrack.com/eamonnduff
Confidence: 0.6599999999999999

To match: ['https://muckrack.com/alessandra-scotto-di-santolo']
Highest match: https://muckrack.com/alessandra-bellotto
Confidence: 0.71

To match: ['https://muckrack.com/ryan-taylor']
Highest match: https://muckrack.com/ryan-taylor-3
Confidence: 0.87

To match: ['https://muckrack.com/liam-doyle']
Highest match: https://muckrack.com/liam-james-doyle
Confidence: 0.76

To match: ['https://muckrack.com/jessica-dolcourt']
Highest match: https://muckrack.com/jessica-domel
Confidence: 0.73

To match: ['https://muckrack.com/unzela-khan']
Highest match: https://muckrack.com/unzela-anna-khan-1
Confidence: 0.85

To match: ['https://muckrack.com/elle-may-rice']
Highest match: https://muckrack.com/emily-rice
Confidence: 0.71

To match: ['https://muckrack.

To match: ['https://muckrack.com/les-bowen']
Highest match: https://muckrack.com/lesbowen
Confidence: 1.0

To match: ['https://muckrack.com/joe-mario-pedersen']
Highest match: https://muckrack.com/joe-pedersen
Confidence: 0.76

To match: ['https://muckrack.com/j-peterson']
Highest match: https://muckrack.com/jp-emerson
Confidence: 0.85

To match: ['https://muckrack.com/larry-dignan']
Highest match: https://muckrack.com/ldignan
Confidence: 0.73

To match: ['https://muckrack.com/ravindra-n-sonavane']
Highest match: https://muckrack.com/ravindra-sonavane
Confidence: 0.94

To match: ['https://muckrack.com/ross-thomson']
Highest match: https://muckrack.com/jess-thomson
Confidence: 0.73

To match: ['https://muckrack.com/noel-baker']
Highest match: https://muckrack.com/noel-baker
Confidence: 1.0

To match: ['https://muckrack.com/michael-gartland']
Highest match: https://muckrack.com/michaelgartland
Confidence: 1.0

To match: ['https://muckrack.com/richard-youle']
Highest match: https://muckra

To match: ['https://muckrack.com/doug-maccash']
Highest match: https://muckrack.com/dougmaccash
Confidence: 1.0

To match: ['https://muckrack.com/gene-maddaus']
Highest match: https://muckrack.com/genemaddaus
Confidence: 1.0

To match: ['https://muckrack.com/tom-whipple']
Highest match: https://muckrack.com/whippletom
Confidence: 0.94

To match: ['https://muckrack.com/colm-o’connor']
Highest match: https://muckrack.com/carolmorton
Confidence: 0.69

To match: ['https://muckrack.com/sinéad-baker']
Highest match: https://muckrack.com/sinead-baker
Confidence: 0.9299999999999999

To match: ['https://muckrack.com/peter-ryan']
Highest match: https://muckrack.com/peter-ryan-3
Confidence: 0.88

To match: ['https://muckrack.com/zoe-wood']
Highest match: https://muckrack.com/zoewoodguardian
Confidence: 0.76

To match: ['https://muckrack.com/dm-martins']
Highest match: https://muckrack.com/ed-martin
Confidence: 0.75

To match: ['https://muckrack.com/lauren-m-johnson']
Highest match: https://muckra

To match: ['https://muckrack.com/james-t-norman']
Highest match: https://muckrack.com/art-norman
Confidence: 0.73

To match: ['https://muckrack.com/larry-lage']
Highest match: https://muckrack.com/larrylage
Confidence: 1.0

To match: ['https://muckrack.com/bill-barrow']
Highest match: https://muckrack.com/billbarrowap
Confidence: 0.9

To match: ['https://muckrack.com/robert-miller']
Highest match: https://muckrack.com/robert-miller-1
Confidence: 0.9299999999999999

To match: ['https://muckrack.com/marc-levy']
Highest match: https://muckrack.com/marc-levine
Confidence: 0.73

To match: ['https://muckrack.com/david-j-mitchell']
Highest match: https://muckrack.com/david-mitchell
Confidence: 0.74

To match: ['https://muckrack.com/justin-fenton']
Highest match: https://muckrack.com/justinfenner
Confidence: 0.83

To match: ['https://muckrack.com/toh-wen-li']
Highest match: https://muckrack.com/tim-rohwer
Confidence: 0.65

To match: ['https://muckrack.com/charles-passy']
Highest match: https:/

To match: ['https://muckrack.com/rickey-ciapha-dennis-jr']
Highest match: https://muckrack.com/rickey-dennis-jr
Confidence: 0.75

To match: ['https://muckrack.com/matthew-j-belvedere']
Highest match: https://muckrack.com/matthew-belvedere
Confidence: 0.71

To match: ['https://muckrack.com/matthew-j-belvedere']
Highest match: https://muckrack.com/matthew-belvedere
Confidence: 0.71

To match: ['https://muckrack.com/carolyn-cohn']
Highest match: https://muckrack.com/carolyn-cox
Confidence: 0.77

To match: ['https://muckrack.com/afdhel-aziz']
Highest match: https://muckrack.com/afdhel-aziz
Confidence: 1.0

To match: ['https://muckrack.com/ryan-o’rourke']
Highest match: https://muckrack.com/ryan-burkett
Confidence: 0.69

To match: ['https://muckrack.com/barney-ronay']
Highest match: https://muckrack.com/barneyronay
Confidence: 1.0

To match: ['https://muckrack.com/april-joyner']
Highest match: https://muckrack.com/april-jackson
Confidence: 0.6599999999999999

To match: ['https://muckrack.co

In [22]:
tm.matcher(original = ["https://muckrack.com/unzela-khan"], lookup = list(muckrack.url), k_matches = 1, ngram_length = 2)['Lookup 1'].iloc[0]

'https://muckrack.com/unzela-anna-khan-1'

In [23]:
tm.matcher(original = ["https://muckrack.com/unzela-khan"], lookup = list(muckrack.url), k_matches = 1, ngram_length = 2)

,Match Confidence,Original Name,Lookup 1
0,0.85,https://muckrack.com/unzela-khan,https://muckrack.com/unzela-anna-khan-1


In [32]:
scores = [0.89, 0.6599999999999999, 0.71, 0.87, 0.76, 0.73, 0.85, 0.71, 0.75, 0.73, 1.0, 0.65, 0.6599999999999999, 0.74, 0.73, 0.81, 0.72, 0.7, 1.0, 0.91, 0.79, 0.76, 0.73, 0.65, 0.84, 0.7, 0.65, 0.8, 1.0, 1.0, 0.89, 0.76, 0.6699999999999999, 0.9, 0.84, 1.0, 1.0, 0.83, 0.85, 0.6799999999999999, 0.87, 0.63, 0.94, 0.73, 1.0, 0.78, 0.75, 0.91, 1.0, 1.0, 0.76, 0.83, 0.85, 1.0, 0.9, 0.6, 0.89, 0.73, 0.96, 0.6799999999999999, 0.71, 0.6699999999999999, 0.84, 0.91, 0.73, 0.73, 0.99, 0.72, 1.0, 0.73, 0.86, 1.0, 0.76, 0.85, 0.73, 0.94, 0.73, 1.0, 1.0, 0.84, 0.9, 0.7, 1.0, 1.0, 0.64, 0.62, 0.71, 0.88, 0.79, 0.56, 1.0, 0.8200000000000001, 0.94, 0.85, 1.0, 0.83, 0.6699999999999999, 0.69, 0.71, 0.77, 1.0, 0.86, 0.83, 0.78, 0.9, 0.61, 0.73, 0.83, 0.84, 0.72, 0.8, 0.8, 1.0, 0.62, 0.75, 0.79, 0.77, 0.6799999999999999, 0.74, 0.75, 0.84, 0.8, 0.72, 0.75, 0.64, 0.76, 0.84, 0.65, 0.92, 0.96, 0.85, 0.9, 1.0, 0.84, 1.0, 0.8, 1.0, 0.95, 0.9, 0.95, 0.84, 0.77, 1.0, 1.0, 0.94, 0.69, 0.9299999999999999, 0.88, 0.76, 0.75, 0.85, 0.91, 0.9299999999999999, 0.96, 0.81, 0.54, 0.9, 0.8200000000000001, 1.0, 0.5700000000000001, 0.79, 0.69, 0.91, 0.83, 0.84, 1.0, 0.9, 1.0, 0.74, 1.0, 0.87, 0.76, 0.91, 1.0, 1.0, 0.87, 1.0, 0.77, 1.0, 0.92, 0.74, 0.9, 0.91, 0.81, 1.0, 0.77, 0.89, 1.0, 0.81, 0.78, 1.0, 1.0, 0.75, 0.71, 0.88, 0.96, 1.0, 1.0, 1.0, 0.71, 0.84, 0.8200000000000001, 1.0, 1.0, 0.69, 0.95, 0.83, 0.63, 0.84, 0.92, 0.72, 0.6799999999999999, 0.73, 1.0, 0.9, 0.9299999999999999, 0.73, 0.74, 0.83, 0.65, 1.0, 0.72, 0.6, 0.5700000000000001, 0.6699999999999999, 0.76, 0.74, 0.69, 1.0, 0.9299999999999999, 0.75, 0.79, 0.87, 1.0, 0.75, 0.81, 1.0, 0.8, 1.0, 0.75, 0.91, 0.73, 0.78, 0.79, 0.76, 0.92, 0.76, 0.72, 0.74, 0.77, 0.78, 0.71, 0.71, 1.0, 0.7, 0.87, 0.76, 1.0, 1.0, 1.0, 1.0, 0.74, 1.0, 1.0, 1.0, 0.71, 0.8200000000000001, 1.0, 0.79, 1.0, 0.78, 1.0, 0.6599999999999999, 0.75, 0.65, 0.84, 0.88, 0.78, 0.88, 1.0, 1.0, 0.74, 0.75, 0.71, 0.71, 0.77, 1.0, 0.69, 1.0, 0.6599999999999999, 1.0, 0.86, 0.69, 0.79, 1.0, 0.6699999999999999, 0.71, 0.8, 0.85, 0.86, 0.6799999999999999, 0.73, 1.0, 0.97, 0.88, 1.0, 0.79, 0.78, 1.0, 0.71, 0.6699999999999999, 0.81, 1.0, 1.0, 0.64, 0.8200000000000001, 0.79, 0.78, 1.0, 0.74, 0.76, 0.76, 0.9299999999999999, 0.69, 0.72, 1.0, 0.53, 0.75, 0.8200000000000001, 0.6699999999999999, 0.79, 0.86, 0.63, 0.74, 0.78, 0.8, 1.0, 0.6599999999999999, 1.0, 0.92, 0.87, 1.0, 0.72, 1.0, 0.76, 0.92, 0.83, 0.95, 0.8, 1.0, 1.0, 0.89]

In [39]:
len([i for i in scores if i >= 0.8])

195